In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = pd.read_csv("bolivia_dataset.csv")  

drop_columns = ["casos acumulados", "incidencia_x_100milHab", "casos_estandarizados", "población", "department", "estacion_met"]
df_cleaned = df.drop(columns=drop_columns, errors='ignore')

df_cleaned = df_cleaned.dropna()
df_cleaned['NDVI'] = df_cleaned['NDVI'].fillna(df_cleaned['NDVI'].mean())
df_cleaned['NINO_3_4'] = df_cleaned['NINO_3_4'].fillna(df_cleaned['NINO_3_4'].mean())

df_cleaned['cases_avg_2w'] = df_cleaned['cases'].rolling(window=2).mean()

df_cleaned = df_cleaned.dropna().reset_index(drop=True)

df_cleaned.to_csv("cleaned_dengue_dataset.csv", index=False)

# Define features (X) and target variable (y)
X = df_cleaned.drop(columns=['cases'])  
y = df_cleaned['cases']  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=38, shuffle=False)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = LinearRegression()
model.fit(X_train_scaled, y_train)

y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"R-squared (R2): {r2}")

default_cases_avg_2w = df_cleaned['cases_avg_2w'].mean()

def predict_dengue_cases(input_data):
    """
    Predict dengue cases given the current weather conditions.
    If past cases are unknown, the model uses the mean of past cases from training data.
    """
    input_data.append(default_cases_avg_2w)

    input_scaled = scaler.transform([input_data])
    
    prediction = model.predict(input_scaled)[0]
    
    return round(prediction, 2)

# Format: [year, epidemiological week, precipitation, maxTemp, minTemp, NDVI, NINO_3_4]
sample_input = [2025, 15, 10.5, 32.0, 22.5, 0.72, 26.0]  
print(f"Predicted dengue cases: {predict_dengue_cases(sample_input)}")

Mean Absolute Error (MAE): 10.962682306232411
Mean Squared Error (MSE): 500.34840319894045
Root Mean Squared Error (RMSE): 22.36846895071141
R-squared (R2): 0.98592877261028
Predicted dengue cases: 91.57


D:\Files\VSC\Learning ML\mlenv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
